In [1]:
from statsmodels.stats.power import TTestIndPower
from math import ceil
power_analysis = TTestIndPower()
sample_size = ceil(power_analysis.solve_power(effect_size=0.5, power=0.8, alpha=0.05))
print(f'Sample Size: {sample_size}')

Sample Size: 64


In [2]:
new_sample = [_ for _ in range(sample_size)]
print(f'{new_sample}')

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]


In [9]:
from multiprocessing.pool import ThreadPool as Pool
# Caution: It's required to use Threadpool if using jupyter notebook.
# As I only did this in jupyter notebook I cannot guarantee this will work in a normal .py file. 
import numpy as np

rng = np.random.default_rng(42)
def generate_sample():
    return rng.uniform(size=sample_size)



s1 = generate_sample()
s2 = generate_sample()
print(f'{s1.mean()}, {s2.mean()}')

pool = Pool()

pool.close()
pool.join()

0.5259108585387304, 0.4520530707552624


In [7]:
import numpy as np
from scipy import stats
from multiprocessing.pool import ThreadPool as Pool

# Set random seed for reproducibility
rng = np.random.default_rng(42)

# Parameters
sample_size = 64
sig = 0.05
reject_count = 0
experiment_count = 0

def increase_experiment_count():
    global experiment_count
    experiment_count += 1

def increase_reject_count():
    global reject_count
    reject_count += 1

def generate_other_sample(sample):
    new_sample = rng.uniform(size=sample_size)
    new_sample = new_sample - new_sample.mean()  # Center the sample around its mean
    new_sample = sample.mean() + (sample.std() / new_sample.std()) * new_sample  # Scale to have the same std as sample1
    new_sample = new_sample + (sample.mean() - new_sample.mean())  # Adjust the mean to match sample1
    return new_sample

def generate_samples(sample_size):
    sample1 = rng.uniform(0, 1, sample_size)
    sample2 = generate_other_sample(sample1)
    return sample1, sample2

def perform_ttest(sample_size, sig):
    s1, s2 = generate_samples(sample_size)
    _, p = stats.ttest_ind(s1, s2)
    return p < sig

def perform_experiment(sample_size, sig):
    increase_experiment_count()
    if perform_ttest(sample_size, sig):
        increase_reject_count()


pool = Pool()
[pool.apply_async(perform_experiment, args=(sample_size, sig)) for _ in range(1000)]
pool.close()
pool.join()
print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')


0.7739560485559633, 0.7157605151013339
0.4388784397520523, 0.6314303207447497
0.8585979199113825, 0.6372794262764648
0.6973680290593639, 0.3719828475034329
0.09417734788764953, 0.08810801835633236
0.9756223516367559, 0.5099720092923378
0.761139701990353, 0.2791025905248099
0.7860643052769538, 0.48067457217344184
0.12811363267554587, 0.9430463588776024
0.45038593789556713, 0.29921864776482543


0.10340296772255164, 0.13508182169379424
0.587644572177712, 0.8556703624412693
0.1705929685368861, 0.16613541874214255
0.9251201183767972, 0.19256817268871868
0.581061139700395, 0.6199383384279561
0.34686980453483707, 0.8998701333211085
0.5909154914814168, 0.2100310057133597
0.022803871029697498, 0.32588695880290297
0.9585592132414453, 0.8010349552431171
0.48230343694290023, 0.9988143451481653


0.8165687634239104, 0.8386148417525947
0.10527807985412496, 0.36794607804772606
0.06655885695517816, 0.20305412618746882
0.5944336637564518, 0.426881995270946
0.14617324419269828, 0.7653426348013304
0.824